# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124727e+02     1.689042e+00
 * time: 0.08120584487915039
     1     1.081341e+01     9.317408e-01
 * time: 1.5418529510498047
     2    -1.162880e+01     1.084074e+00
 * time: 1.631497859954834
     3    -3.388843e+01     8.365345e-01
 * time: 1.748384952545166
     4    -4.733366e+01     5.965129e-01
 * time: 1.9098739624023438
     5    -5.671839e+01     2.021722e-01
 * time: 2.007145881652832
     6    -5.963250e+01     1.403790e-01
 * time: 2.0826778411865234
     7    -6.075847e+01     5.135033e-02
 * time: 2.1718249320983887
     8    -6.118526e+01     6.484522e-02
 * time: 2.246706962585449
     9    -6.148987e+01     4.938019e-02
 * time: 2.3247568607330322
    10    -6.173955e+01     3.845147e-02
 * time: 2.402040958404541
    11    -6.194894e+01     2.121933e-02
 * time: 2.4865598678588867
    12    -6.203804e+01     1.879116e-02
 * time: 2.564116954803467
    13    -6.213461e+01     1.816842e-02
 * time: 2.642071008682251

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557674
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336817

    total               -62.261666461693
